## Combinatorial Optimization

#### Assignment IV

***Heuristics for the Knapsack Problem - Solving Multiple Instances***

**Instance:** C5I10

Guilherme Cadori

09/07/2023


In [1]:
# Importing required libraries
import numpy as np


#### 1. Read the instance

In [2]:
# Reading the target file
# GC: the code template below was provided in class and was adapted by GC (2023-07-09)

filename = "C:\gccadori\Lab4_CO\Knap_C5I10.dat"  # Substitua pelo nome do arquivo real

# Variáveis para armazenar os dados lidos
linha = None
coluna = None
cost = None
rhs = None
matrix = None

# Função auxiliar para converter uma linha de valores em uma lista de floats
def parse_values(line):
    return [float(value) for value in line.strip().split()]

# Ler o arquivo e extrair os dados
with open(filename, "r") as file:
    lines = file.readlines()

# Ler o escalar
linha = int(lines[0].strip())
coluna = int(lines[2].strip())

# Ler o vetor
cost = np.array(parse_values(lines[5]))

# Ler a matriz
end = 7 + linha
matrix_lines = [line.strip() for line in lines[7:end]]
matrix = np.array([parse_values(line) for line in matrix_lines],dtype=object)

# Ler vetor de RHS
rhs = np.array(parse_values(lines[end+1]))

# Imprimir os dados lidos
print("\n linhas:", linha)
print("\n colunas:", coluna)
print("\n cost:", cost)
print("\n rhs:", rhs)
print("\n Matrix:")
for row in matrix:
    print(row)



 linhas: 5

 colunas: 10

 cost: [0.30843 0.74907 0.6691  0.52619 0.61796 0.07105 0.16246 0.39898 0.57834
 0.19578]

 rhs: [116.12312  82.05387 109.88891  69.12963 128.29675]

 Matrix:
[13.2553 6.76342 22.86785 14.97223 13.27365 15.19261 17.27234 21.22448
 19.85637 0.93261]
[1.32876 7.51181 1.00781 13.34117 0.11301 24.33323 18.55862 14.78065
 18.35194 16.00987]
[17.40859 11.14715 0.07718 22.18725 1.06069 1.94107 23.637 17.02113
 24.08336 19.02182]
[0.27528 1.69006 4.87256 18.16527 3.74127 4.52415 12.46014 17.57698
 16.24719 16.45475]
[15.11814 15.51384 16.12041 10.67837 21.38512 19.73834 17.26438 12.49316
 22.49747 15.0008]


#### 2.	Construct an initial feasible solution

In [3]:
# Renaming variables for testing purposes
c = cost

A = matrix

b = rhs


In [4]:
# Providing an initial feasible solution

def randomStart(nVars, A, b, c):
    # Creating variable names x1, ..., x_nVars
    variable_names = ['x{}'.format(i) for i in range(1, nVars+1)]

    while True:
        np.random.seed(12345)
        x = np.random.randint(2, size=nVars)
        
        if np.all(A @ x <= b):
            FO = x @ c
            print('The initial feasible solution is:')
            for i, var in enumerate(variable_names):
                print('Item {}: {}'.format(i+1, x[i]))   
            print('Objective function value:', round(FO, 2))
            return


In [5]:
# Testing the function
randomStart(nVars = coluna, A = A, b = b, c = c)


The initial feasible solution is:
Item 1: 0
Item 2: 1
Item 3: 1
Item 4: 1
Item 5: 0
Item 6: 1
Item 7: 0
Item 8: 0
Item 9: 1
Item 10: 0
Objective function value: 2.59


#### 3.	Program the heuristic you proposed in the previous assingment

In [6]:
# Implementing the heuristic algorithm proposed in Lab III

def naiveKnapsack(nVars, items_c, capacity_b, A):
    # Sort the items in descending order of value
    sorted_items = sorted(range(nVars), key=lambda i: items_c[i], reverse=True)

    # Initialize variables
    selected_items = np.zeros(nVars, dtype=int)
    total_weight = np.zeros_like(capacity_b, dtype=items_c.dtype)
    total_value = 0

    # Greedy selection process
    for idx in sorted_items:
        weight = A[:, idx]
        if np.all(total_weight + weight <= capacity_b):
            # Add the item to the knapsack
            selected_items[idx] = 1
            total_weight += weight.astype(total_weight.dtype)  # Ensure matching data type
            total_value += items_c[idx]

    # Print the selected items and total value
    print('The initial feasible solution is:')
    for idx in range(nVars):
        print('Item {}: {}'.format(idx + 1, selected_items[idx]))
    print('Objective function value:', round(total_value, 2))

    # Return the complete list of items
    return


In [7]:
# Testing the function
naiveKnapsack(nVars = coluna, items_c = c, capacity_b = b , A = A)


The initial feasible solution is:
Item 1: 1
Item 2: 1
Item 3: 1
Item 4: 1
Item 5: 1
Item 6: 0
Item 7: 0
Item 8: 1
Item 9: 1
Item 10: 0
Objective function value: 3.85


#### 4.	Program a Local Search 

In [8]:
# Local Search metaheuristic implementation
def LocalSearch(nVars, A, b, c, nStarts=1, max_iterations=1000):
    import numpy as np

    # Creating variable names
    variable_names = ['x{}'.format(i) for i in range(1, nVars + 1)]

    best_solution = None
    best_value = -np.inf

    # Escaping Local Optima - Multiple Starting Solutions
    for n in range(1, nStarts + 1):
        # Producing an initial feasible solution
        while True:
            x = np.random.randint(2, size=nVars)
            if np.all(A @ x <= b):
                FO = x @ c
                break  # Exit the loop once an initial feasible solution is found

        # Setting initial search parameters
        s = FO
        s_new = 0
        n_iterations = 0
        best_iteration = 0

        # Termination criteria
        while n_iterations - best_iteration < max_iterations:
            n_iterations += 1

            # Neighbor Selection: Swap Two - First Improvement Strategy
            def generate_neighbor_solution(x):
                x_new = x.copy()
                idx1, idx2 = np.random.choice(len(x), size=2, replace=False)
                x_new[idx1], x_new[idx2] = x_new[idx2], x_new[idx1]
                return x_new

            # Generating neighbor solution
            x_new = generate_neighbor_solution(x)

            # Evaluate solution
            if np.all(A @ x_new <= b):
                FO_new = x_new @ c
            else:
                FO_new = -np.inf

            # Update the current solution
            if FO_new > s_new:
                x = x_new
                s_new = FO_new
                best_iteration = n_iterations

                # Check if the current solution is the best found so far
                if s_new > best_value:
                    best_solution = x.copy()
                    best_value = s_new

    # Print the best solution found
    print('The best solution found:')
    print('Objective function value:', round(best_value, 2))

    return


#### 5.	Running the heuristic for the given instance

In [9]:
import time

# Record starting time
start_time = time.time()

# Running the proposed heuristic for this instance
LocalSearch(nVars = coluna, A = A, b = b, c = c, 
            nStarts = 50,
            max_iterations = 100)

# Record ending time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = round(end_time - start_time, 2)

# Print the elapsed time
print("Elapsed time:", elapsed_time, "seconds")


The best solution found:
Objective function value: 3.85
Elapsed time: 0.33 seconds


**Test Run Results**

The best solution found:

- Objective function value: 3.85
- Elapsed time: 0.33 seconds


### End